# Sistema de Reconocimento de Lenguaje de Señas

El proyecto consiste en la realización de un sistema de aprendizaje de lenguaje de señas. En el cual el usuario mediante el uso de la aplicación podrá agilizar su aprendizaje en el área ya mencionada.

Realizado por:
    
    Sebastián Medina
    Diego Orellana
    Jorge Peralta
    Ronnie Urdiales

# Importación de librerías

In [ ]:
import cv2
import os
import time
import uuid
import numpy as np

import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

# Toma de capturas

In [ ]:
ruta_imagenes = 'Tensorflow/workspace/images/collectedimages'
frases = ['hola', 'gracias', 'si', 'no', 'tequiero','A','E','I','O','U']
num_img = 15 #numero de fotos a tomar

# captura de fotos
for frase in frases:
    !mkdir {'Tensorflow\workspace\images\collectedimages\\'+frase}
    captura = cv2.VideoCapture(0)
    print('Tomando fotos para {}'.format(frase))
    time.sleep(5)
    for imgnum in range(num_img):
        ret, frame = captura.read()
        imgname = os.path.join(ruta_imagenes, frase, frase+'.'+'{}.jpg'.format(str(uuid.uuid1())))
        cv2.imwrite(imgname, frame)
        cv2.imshow('frame', frame)
        
        time.sleep(2)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    captura.release()

# Rutas

In [1]:
RUTA_TRABAJO = 'Tensorflow/workspace'
RUTA_SCRIPTS = 'Tensorflow/scripts'
RUTA_MODELO_API = 'Tensorflow/models'
RUTA_ANOTACIONES = RUTA_TRABAJO+'/annotations'
RUTA_IMAGENES = RUTA_TRABAJO+'/images'
RUTA_MODELOS = RUTA_TRABAJO+'/models'
RUTA_MODELOS_ENT = RUTA_TRABAJO+'/pre-trained-models'
RUTA_CONF = RUTA_MODELOS+'/my_ssd_mobnet/pipeline.config'
RUTA_CHECKPOINT = RUTA_MODELOS+'/my_ssd_mobnet/'
NOMBRE_MODELO = 'my_ssd_mobnet' 

# 1.- Palabras a reconocer

In [3]:
labels = [{'name':'hola', 'id':1},
         {'name':'si', 'id':2},
         {'name':'no', 'id':3},
         {'name':'gracias', 'id':4},
         {'name':'tequiero', 'id':5},
         {'name':'A', 'id':6},
         {'name':'E', 'id':7},
         {'name':'I', 'id':8},
         {'name':'O', 'id':9},
         {'name':'U', 'id':10}]

with open(RUTA_ANOTACIONES + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 2.- Creación de los modelos a entrenar

In [3]:
!python {RUTA_SCRIPTS + '/generate_tfrecord.py'} -x {RUTA_IMAGENES + '/train'} -l {RUTA_ANOTACIONES + '/label_map.pbtxt'} -o {RUTA_ANOTACIONES + '/train.record'}
!python {RUTA_SCRIPTS + '/generate_tfrecord.py'} -x{RUTA_IMAGENES + '/test'} -l {RUTA_ANOTACIONES + '/label_map.pbtxt'} -o {RUTA_ANOTACIONES + '/test.record'}

!mkdir {'Tensorflow\workspace\models\\' + NOMBRE_MODELO}
!cp {RUTA_MODELOS_ENT+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {RUTA_MODELOS+'/'+NOMBRE_MODELO}

!cd Tensorflow && git clone https://github.com/tensorflow/models #descarga de modelos

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


# 3.- Configuración preparatoria para el entrenamiento

In [4]:
RUTA_CONF = RUTA_MODELOS+'/'+NOMBRE_MODELO+'/pipeline.config'
config = config_util.get_configs_from_pipeline_file(RUTA_CONF)

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(RUTA_CONF, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  
    

pipeline_config.model.ssd.num_classes = 5
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = RUTA_MODELOS_ENT+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= RUTA_ANOTACIONES + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [RUTA_ANOTACIONES + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = RUTA_ANOTACIONES + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [RUTA_ANOTACIONES + '/test.record']

config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(RUTA_CONF, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 4.- Generar comando para entrenar el modelo

In [16]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=5000""".format(RUTA_MODELO_API, RUTA_MODELOS,NOMBRE_MODELO,RUTA_MODELOS,NOMBRE_MODELO))

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=5000


# 5.- Carga del modelo entrenado

In [11]:
configs = config_util.get_configs_from_pipeline_file(RUTA_CONF)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(RUTA_CHECKPOINT, 'ckpt-6')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 6.- Funcionamiento

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(RUTA_ANOTACIONES+'/label_map.pbtxt')

cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=3,
                min_score_thresh=.9,
                agnostic_mode=False)

    cv2.imshow('Sistema de reconocimiento de señas',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        #cap.release()
        break

In [1]:
cap.release() #finalizar sesión de la cámara